In [3]:
import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import keras
import random
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D, CuDNNLSTM, Flatten
from keras.layers import Lambda, AveragePooling1D, MaxPooling1D, Bidirectional, GlobalMaxPool1D, Concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D,concatenate
from keras.layers import SpatialDropout1D
from keras.models import Sequential
from keras.callbacks import Callback
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.layers import Input, Add, Dense, Activation, ZeroPadding1D, BatchNormalization, Flatten, Conv1D, AveragePooling1D, MaxPooling1D, GlobalMaxPooling1D
from keras.initializers import glorot_uniform
from keras.models import Model, load_model
from keras.optimizers import SGD,Adam

In [4]:
import os
import warnings
from keras import backend as K
warnings.filterwarnings('ignore')

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
random.seed(12345)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
tf.set_random_seed(1234)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [4]:
from keras.regularizers import l2


def W_init(shape,name=None):
    """Initialize weights as in paper"""
    values = np.random.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values,name=name)
#//TODO: figure out how to initialize layer biases in keras.
def b_init(shape,name=None):
    """Initialize bias as in paper"""
    values=np.random.normal(loc=0.5,scale=1e-2,size=shape)
    return K.variable(values,name=name)

input_shape = (1001, 1)
left_input = Input(input_shape)
right_input = Input(input_shape)
#build convnet to use in each siamese 'leg'
convnet = Sequential()
convnet.add(Conv1D(64,10,activation='relu',input_shape=input_shape,
                   kernel_initializer=W_init,kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling1D())
convnet.add(Conv1D(128,7,activation='relu',
                   kernel_regularizer=l2(2e-4),kernel_initializer=W_init,bias_initializer=b_init))
convnet.add(MaxPooling1D())
convnet.add(Conv1D(128,4,activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
convnet.add(MaxPooling1D())
convnet.add(Conv1D(256,4,activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
convnet.add(Flatten())
convnet.add(Dense(4096,activation="sigmoid",kernel_regularizer=l2(1e-3),kernel_initializer=W_init,bias_initializer=b_init))

#call the convnet Sequential model on each of the input tensors so params will be shared
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)
#layer to merge two encoded inputs with the l1 distance between them
L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
#call this layer on list of two input tensors.
L1_distance = L1_layer([encoded_l, encoded_r])
prediction = Dense(1,activation='sigmoid',bias_initializer=b_init)(L1_distance)
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

optimizer = Adam(0.00006)
#//TODO: get layerwise learning rates and momentum annealing scheme described in paperworking
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)

siamese_net.count_params()

Instructions for updating:
Colocations handled automatically by placer.


123995329

In [ ]:
file_to_read = open('new_data.pickle', 'rb')

#通过pickle的load函数读取data1.pkl中的对象，并赋值给data2
tmp = pickle.load(file_to_read)

y_train = pd.get_dummies(Data.iloc[all_index,:]['label'])


In [ ]:
cnn_model = Sequential()
cnn_model.add(Conv1D(6, 5, strides = 1, activation = 'relu', input_shape = (1001, 1)))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))
cnn_model.add(Conv1D(16, 5, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))
cnn_model.add(Conv1D(32, 5, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(120, activation = 'relu'))
cnn_model.add(Dense(84, activation = 'relu'))
cnn_model.add(Dense(186, activation = 'softmax', activity_regularizer = keras.regularizers.l2(0.1)))
cnn_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [ ]:
cnn_model.fit(X_train.reshape(118880, 1001, 1), y_train.values, batch_size = 64, epochs = 100, verbose = 1)

## 搭建SiameseNet模型
基本上是加深版本的Lenet 5

In [5]:
input_shape = (2251, 1)
left_input = Input(input_shape)
right_input = Input(input_shape)
#build convnet to use in each siamese 'leg'

# Construct CNN layers

# cnn_model = Sequential()
# cnn_model.add(Conv1D(64, 3, strides = 1, activation = 'relu', input_shape = (2251, 1)))
# cnn_model.add(Conv1D(64, 3, strides = 1, activation = 'relu'))
# cnn_model.add(MaxPooling1D(2, strides = 2))
# cnn_model.add(Dropout(0.2))

# cnn_model.add(Conv1D(128, 3, strides = 1, activation = 'relu'))
# cnn_model.add(Conv1D(128, 3, strides = 1, activation = 'relu'))
# cnn_model.add(MaxPooling1D(2, strides = 2))
# cnn_model.add(Dropout(0.2))

# cnn_model.add(Conv1D(256, 3, strides = 1, activation = 'relu'))
# # cnn_model.add(Conv1D(256, 3, strides = 1, activation = 'relu'))
# cnn_model.add(MaxPooling1D(2, strides = 2))
# cnn_model.add(Dropout(0.2))

# cnn_model.add(Conv1D(512, 3, strides = 1, activation = 'relu'))
# #cnn_model.add(Conv1D(512, 3, strides = 1, activation = 'relu'))
# cnn_model.add(MaxPooling1D(2, strides = 2))
# cnn_model.add(Dropout(0.2))

# cnn_model.add(Flatten())
# cnn_model.add(Dense(120, activation = 'relu'))
# cnn_model.add(Dense(84, activation = 'relu'))




# Lenet 5

cnn_model = Sequential()
cnn_model.add(Conv1D(6, 5, strides = 1, activation = 'relu', input_shape = (2251, 1)))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(16, 5, strides = 1, activation = 'relu'))
cnn_model.add(Conv1D(16, 5, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(32, 5, strides = 1, activation = 'relu'))
cnn_model.add(Conv1D(32, 5, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(64, 5, strides = 1, activation = 'relu'))
cnn_model.add(Conv1D(64, 5, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Flatten())
cnn_model.add(Dense(120, activation = 'relu'))
cnn_model.add(Dense(84, activation = 'relu'))
cnn_model.add(Dense(186, activation = 'softmax', activity_regularizer = keras.regularizers.l2(0.1)))

#call the convnet Sequential model on each of the input tensors so params will be shared
encoded_l = cnn_model(left_input)
encoded_r = cnn_model(right_input)
#layer to merge two encoded inputs with the l1 distance between them
L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
#call this layer on list of two input tensors.
L1_distance = L1_layer([encoded_l, encoded_r])
#prediction = Dense(1,activation='sigmoid',bias_initializer=b_init)(L1_distance)
prediction = Dense(1,activation='sigmoid')(L1_distance)
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

optimizer = Adam(0.00016)
#//TODO: get layerwise learning rates and momentum annealing scheme described in paperworking
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)

siamese_net.count_params()

1088141

In [7]:
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_8 (Conv1D)            (None, 2247, 6)           36        
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 1123, 6)           0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 1123, 6)           0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 1119, 16)          496       
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 1115, 16)          1296      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 557, 16)           0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 557, 16)           0         
__________

In [6]:
siamese_net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 2251, 1)      0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 2251, 1)      0                                            
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 186)          1087954     input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 186)          0           sequential_2[1][0]               
          

In [23]:
# file_to_read = open('train_pairs.pickle', 'rb')

# train_pairs = pickle.load(file_to_read)



# file_to_read = open('train_targets.pickle', 'rb')

# train_targets = pickle.load(file_to_read)


file_to_read = open('test_pairs.pickle', 'rb')

test_pairs = pickle.load(file_to_read)


file_to_read = open('test_targets.pickle', 'rb')

test_targets = pickle.load(file_to_read)


In [12]:
# new train data
#打开一个名为data1.pkl的文件，打开方式为二进制读取(参数‘rb’)
file_to_read = open('new_data.pickle', 'rb')

#通过pickle的load函数读取data1.pkl中的对象，并赋值给data2
tmp = pickle.load(file_to_read)

# train_data = tmp  
t = np.array(tmp)
t.shape


(2418, 2252)

In [5]:
train_targets, train_pairs = [],[]
pair_1, pair_2 = [], []
merge_label_pair = []

# label 1 pairs
for n in range(186):
    locfix = n*13
    for i in range(13):
        loc = n*13+i
        for j in range(i+1, 13):
            merge_label_pair.append(train_data[loc][:-1])
            merge_label_pair.append(train_data[locfix+j][:-1])
            merge_label_pair.append(1)

# merge_label_pair = np.array(merge_label_pair)
# print(merge_label_pair.shape)            
# print(merge_label_pair[2])            
# merge_label_pair = np.array(merge_label_pair).reshape(186*78,3)
# print(merge_label_pair.shape)



# label 0 pairs

for n in range(186):
    locfix = n*13
    index = np.arange(2418)
    delete_index = list(range(n*13,(n+1)*13))
    rand_index = np.random.choice(np.delete(index,delete_index,0),6)
    for i in range(13):
        loc = n*13+i
        for j in range(6):
            merge_label_pair.append(train_data[loc][:-1])
            merge_label_pair.append(train_data[rand_index[j]][:-1])
            merge_label_pair.append(0)

merge_label_pair = np.array(merge_label_pair)
print(merge_label_pair.shape)

        
merge_label_pair = merge_label_pair.reshape(186*78*2,3)
print(merge_label_pair.shape)

# print(merge_label_pair[1][1])
# a = np.array(merge_label_pair[:][1])
# print(a.shape)

np.random.shuffle(merge_label_pair)

#split
pair_1, pair_2 = [], []
for i in range(29016):
    pair_1.append(merge_label_pair[i][0])
    pair_2.append(merge_label_pair[i][1])

train_pairs = [pair_1, pair_2]

# print(train_pairs)
train_pairs = np.array(train_pairs)
print(train_pairs.shape)

train_pairs = train_pairs.reshape(2,186*78*2,2251,1)
print(train_pairs.shape)
train_pairs = train_pairs.tolist()


train_targets = merge_label_pair[:, -1]


# print(train_targets[1])
print(train_targets.shape)


train_targets = train_targets.tolist()


# print(train_targets)

(87048,)
(29016, 3)
(2, 29016, 2251)
(2, 29016, 2251, 1)
(29016,)


In [ ]:
file_to_read = open('train_low_None_.pickle', 'rb')

tmp1 = pickle.load(file_to_read)

train_data = tmp1.values  


train_targets, train_pairs = [],[]
pair_1, pair_2 = [], []

index = np.arange(800)
# print(index)

rand_index = np.random.choice(index,185)
# print(rand_index)
for i in range(185):
    for j in range(185):
        pair_1.append(train_data[rand_index[i]][:-1])
        pair_2.append(train_data[rand_index[j]][:-1])
        train_targets.append(1)
        
        pair_1.append(train_data[rand_index[i]][:-1])
        pair_2.append(train_data[rand_index[j]+800*(j+1)][:-1])
        train_targets.append(0)
            
            
#print(pairs,targets)
train_pairs = [pair_1, pair_2]
train_pairs = np.array(train_pairs)
print(train_pairs.shape)
train_pairs = train_pairs.reshape(2,185*185*2,1001,1)
print(train_pairs)
print(train_pairs.shape)


# print(train_pairs[:][:10][:10][:])
# train_pairs = train_pairs.tolist()

In [23]:
output = open('train_pairs.pickle', 'wb')

pickle.dump(train_pairs,output)

output.close()

In [28]:
output = open('train_targets.pickle', 'wb')

pickle.dump(train_targets,output)

output.close()

In [36]:

print(train_pairs[0][0][-1][:])

print(train_pairs[1][0][-1][:])

print(train_targets[1])

RASGAT
RASGAT
0


In [5]:
siamese_net.fit(train_pairs, train_targets, batch_size = 64, epochs = 100, verbose = 1)
#cnn_model.fit(X_train.reshape(118880, 1001, 1), y_train.values, batch_size = 64, epochs = 100, verbose = 1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
29016/29016 [==============================] - 11s 366us/step - loss: 0.7589
Epoch 2/100
29016/29016 [==============================] - 8s 269us/step - loss: 0.7503
Epoch 3/100
29016/29016 [==============================] - 8s 273us/step - loss: 0.7357
Epoch 4/100
29016/29016 [==============================] - 8s 281us/step - loss: 0.7168
Epoch 5/100
29016/29016 [==============================] - 8s 277us/step - loss: 0.6976
Epoch 6/100
29016/29016 [==============================] - 8s 275us/step - loss: 0.6792
Epoch 7/100
29016/29016 [==============================] - 8s 269us/step - loss: 0.6620
Epoch 8/100
29016/29016 [==============================] - 8s 270us/step - loss: 0.6457
Epoch 9/100
29016/29016 [==============================] - 8s 272us/step - loss: 0.6295
Epoch 10/100
29016/29016 [==============================] - 8s 274us/step - loss: 0.6134
Epoch 11/100
29016/29016 [==============================] - 8s 275us/s

29016/29016 [==============================] - 8s 276us/step - loss: 0.1564
Epoch 91/100
29016/29016 [==============================] - 8s 277us/step - loss: 0.1548
Epoch 92/100
29016/29016 [==============================] - 8s 277us/step - loss: 0.1538
Epoch 93/100
29016/29016 [==============================] - 8s 277us/step - loss: 0.1526
Epoch 94/100
29016/29016 [==============================] - 8s 277us/step - loss: 0.1513
Epoch 95/100
29016/29016 [==============================] - 8s 277us/step - loss: 0.1503
Epoch 96/100
29016/29016 [==============================] - 8s 277us/step - loss: 0.1492
Epoch 97/100
29016/29016 [==============================] - 8s 277us/step - loss: 0.1483
Epoch 98/100
29016/29016 [==============================] - 8s 277us/step - loss: 0.1471
Epoch 99/100
29016/29016 [==============================] - 8s 276us/step - loss: 0.1462
Epoch 100/100
29016/29016 [==============================] - 8s 277us/step - loss: 0.1449


# 创立训练数据

In [72]:
import numpy as np
import pickle
#trainfile="/home/suliangbu/work/wanghong/train_low_None_.pickle"
file_to_read = open('train_low_None_.pickle', 'rb')
tmp = pickle.load(file_to_read)
data = tmp.values

featur = np.zeros((186, 800, 1001))
for i in range(186):
    featur[i] = data[i * 800:(i+1) * 800, :1001]

def get_batch(batch_size,featur):
    """Create batch of n pairs, half same class, half different class"""
    n_examples = 800
    n_classes = 186

    
    categories = np.random.choice(n_classes,size=(batch_size,),replace=False)
    
    pairs = [np.zeros((batch_size, 1001, 1)) for i in range(2)]
    
    targets=np.zeros((batch_size))

    targets[batch_size//2:] = 1
    
    for i in range(batch_size):

        category = categories[i]
        idx_1 = np.random.randint(0, n_examples)        
        pairs[0][i] = featur[category, idx_1].reshape(1001,1)

        idx_2 = np.random.randint(0, n_examples)

        if i >= batch_size // 2:
            category_2 = category  
        else: 
            category_2 = (category + np.random.randint(1,n_classes)) % n_classes

        pairs[1][i] = featur[category_2,idx_2].reshape(1001,1)
    #pairs = np.array(pairs)
    
    #print(pairs.shape)
    return pairs, targets

(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 1)
(2, 32, 1001, 

In [4]:
file_to_read = open('T_origin.pickle', 'rb')

tmp = pickle.load(file_to_read)

test_data = tmp.values

print(test_data[:,-1])
print(test_data.shape)

['ZIF-67' 'ZIF-67' 'ZIF-67' 'ZIF-67' 'ZIF-71' 'ZIF-71' 'ZIF-71' 'ZIF-8'
 'ZIF-8' 'ZIF-8' 'ZIF-8' 'ZIF-90' 'ZIF-90' 'ZIF-90']
(14, 2252)


In [4]:
test_data = np.load('mix_testdata_and_label.npy')

In [5]:
test_targets, test_pairs = [], []
pair_1,pair_2 = [], []
for i in range(14):
    for j in range(i + 1, 14):
        if test_data[i][-1] == test_data[j][-1]:
            pair_1.append(test_data[i][:-1])
            pair_2.append(test_data[j][:-1])
            #test_pairs.append([test_data[i][:-1], test_data[j][:-1]])
            test_targets.append(1)
        else:
            #test_pairs.append([test_data[i][:-1], test_data[j][:-1]])
            pair_1.append(test_data[i][:-1])
            pair_2.append(test_data[j][:-1])
            test_targets.append(0)
test_pairs = [pair_1,pair_2]
test_pairs = np.array(test_pairs).reshape(2,91,2251,1)
test_pairs = test_pairs.tolist()

In [29]:
output = open('test_pairs.pickle', 'wb')

pickle.dump(test_pairs,output)

output.close()

output1 = open('test_targets.pickle', 'wb')

pickle.dump(test_targets,output1)

output1.close()

# 创立测试数据

In [6]:
file_to_read = open('test__None_.pickle', 'rb')

tmp = pickle.load(file_to_read)
print(tmp)
test_data = tmp.values  
test_data

array([[0.033514307811291076, 0.02062374323279156, 0.027068832173240076,
        ..., 0.037638437741685626, 0.054137664346480555, 'ZIF-67'],
       [0.03475991590433889, 0.04274393096490252, 0.02536397301273244,
        ..., 0.569748107565793, 0.6209495144208277, 'ZIF-67'],
       [0.032538760318275455, 0.04601976130386533, 0.042300527867580276,
        ..., 0.036722374982263904, 0.043695066088909475, 'ZIF-67'],
       ...,
       [0.01178765033802156, 0.013248283739033317, 0.0131714002046804,
        ..., 0.0027931685785534794, 0.0014350260564532555, 'ZIF-90'],
       [0.09375007508678823, 0.11250007065970596, 0.10717590525013387,
        ..., 0.014351836889150319, 0.02152786991703112, 'ZIF-71'],
       [0.11164413672968, 0.11295253299787038, 0.10989987003826307, ...,
        0.02224155251981437, 0.02616669553280636, 'ZIF-71']], dtype=object)

In [7]:
test_targets, test_pairs = [], []
pair_1,pair_2 = [], []
for i in range(14):
    for j in range(i + 1, 14):
        if test_data[i][-1] == test_data[j][-1]:
            pair_1.append(test_data[i][:-1])
            pair_2.append(test_data[j][:-1])
            #test_pairs.append([test_data[i][:-1], test_data[j][:-1]])
            test_targets.append(1)
        else:
            #test_pairs.append([test_data[i][:-1], test_data[j][:-1]])
            pair_1.append(test_data[i][:-1])
            pair_2.append(test_data[j][:-1])
            test_targets.append(0)
test_pairs = [pair_1,pair_2]
# test_pairs = np.array(test_pairs)
# print(test_pairs.shape)
test_pairs = np.array(test_pairs).reshape(2,91,1002,1)
test_pairs = test_pairs.tolist()
#test_targets = np.array(test_targets)
#test_pairs.shape
#print(pairs,targets)

ValueError: cannot reshape array of size 409682 into shape (2,91,1002,1)

In [21]:
print(test_pairs[0][3][-1][:])
print(test_pairs[1][3][-1][:])
print(test_targets[3])

['ZIF-67']
['ZIF-71']
0


# 训练

In [67]:
batch_size = 32
loss_every = 500
for i in range(1,50000):
    (inputs,targets)=get_batch(batch_size,featur)
    loss=siamese_net.train_on_batch(inputs,targets)
    #print(loss)
    if i % loss_every == 0:
        print("iteration {}, training loss: {:.2f},".format(i,loss))


iteration 500, training loss: 0.37,
iteration 1000, training loss: 0.38,
iteration 1500, training loss: 0.40,
iteration 2000, training loss: 0.35,
iteration 2500, training loss: 0.40,
iteration 3000, training loss: 0.30,
iteration 3500, training loss: 0.30,
iteration 4000, training loss: 0.31,
iteration 4500, training loss: 0.33,
iteration 5000, training loss: 0.32,
iteration 5500, training loss: 0.28,
iteration 6000, training loss: 0.26,
iteration 6500, training loss: 0.30,
iteration 7000, training loss: 0.28,
iteration 7500, training loss: 0.23,
iteration 8000, training loss: 0.30,
iteration 8500, training loss: 0.31,
iteration 9000, training loss: 0.23,
iteration 9500, training loss: 0.35,
iteration 10000, training loss: 0.27,
iteration 10500, training loss: 0.24,
iteration 11000, training loss: 0.23,
iteration 11500, training loss: 0.23,
iteration 12000, training loss: 0.27,
iteration 12500, training loss: 0.22,
iteration 13000, training loss: 0.20,
iteration 13500, training loss: 

In [6]:
probs = siamese_net.predict(test_pairs)
n_correct = 0
print(probs)
for i in range(len(probs)):
    if probs[i] <= 0.5:
        probs[i] = 0 
    else:
        probs[i] = 1
print(probs)

for i in range(len(probs)):
    if probs[i] == test_targets[i]:
        n_correct += 1 
percent_correct = (100.0*n_correct / 91)
print(percent_correct)
    
    
    
#    if np.argmax(probs) == np.argmax(test_targets):
#        n_correct+=1
#        percent_correct = (100.0*n_correct / 91)
#print(percent_correct)
#print(len(probs))

[[0.908702  ]
 [0.67001295]
 [0.955983  ]
 [0.7392195 ]
 [0.689074  ]
 [0.6867306 ]
 [0.46435875]
 [0.49303654]
 [0.3723563 ]
 [0.43139482]
 [0.33008108]
 [0.35192215]
 [0.90542334]
 [0.43762007]
 [0.8949132 ]
 [0.52020395]
 [0.45589662]
 [0.4579588 ]
 [0.3002059 ]
 [0.31243443]
 [0.17407334]
 [0.3856165 ]
 [0.28850293]
 [0.30817986]
 [0.9666454 ]
 [0.7373388 ]
 [0.8235408 ]
 [0.87644434]
 [0.87577933]
 [0.87243366]
 [0.87154067]
 [0.8699995 ]
 [0.7475954 ]
 [0.70706505]
 [0.71304613]
 [0.4359071 ]
 [0.79201925]
 [0.74945396]
 [0.74882287]
 [0.54229724]
 [0.5586129 ]
 [0.43171555]
 [0.48004955]
 [0.38380855]
 [0.403131  ]
 [0.8919996 ]
 [0.9500402 ]
 [0.954239  ]
 [0.60935235]
 [0.60994065]
 [0.776379  ]
 [0.4014426 ]
 [0.3447327 ]
 [0.33781812]
 [0.5121361 ]
 [0.9669719 ]
 [0.71483064]
 [0.7164275 ]
 [0.85543597]
 [0.51757944]
 [0.45765305]
 [0.4511677 ]
 [0.45006043]
 [0.70925546]
 [0.71246177]
 [0.84416074]
 [0.51413196]
 [0.45352882]
 [0.445031  ]
 [0.45454037]
 [0.9520977 ]
 [0.85

In [7]:
print(percent_correct)

51.64835164835165


In [8]:
# print(probs)
for i in range(91):
    if(probs[i]==1):
        print(i)

0
1
2
3
4
5
12
14
15
24
25
26
27
28
29
30
31
32
33
34
36
37
38
39
40
45
46
47
48
49
50
54
55
56
57
58
59
63
64
65
66
70
71
72
73
74
76
77
78
79
81
82
83
85
86
88


In [27]:
for i in range(14):
    print(test_data[i][-1])

NameError: name 'test_data' is not defined

In [35]:
for i in range(91):
    if(test_targets[i]==1):
        print(i)

0
1
2
13
14
25
46
47
55
70
71
72
76
77
81
88
89
90


In [7]:
# import h5py

# model = siamese_net
# model.save('model_12_18.h5')

In [3]:
# file_to_read = open('new_test_pairs.pickle', 'rb')

# test_pairs = pickle.load(file_to_read)


# file_to_read = open('new_test_targets.pickle', 'rb')

# test_targets = pickle.load(file_to_read)


from keras.models import load_model
 
siamese_net= load_model('model_12_18.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [6]:
print(test_pairs)
test_pairs = test_pairs.tolist()
print(test_pairs)

[[array([0.0, 0.0, 0.0, ..., 8.27511, 6.47198, 5.21193], dtype=object), array([0.0, 0.0, 0.0, ..., 8.27511, 6.47198, 5.21193], dtype=object), array([0.0, 0.0, 0.0, ..., 8.27511, 6.47198, 5.21193], dtype=object), array([0.0, 0.0, 0.0, ..., 8.27511, 6.47198, 5.21193], dtype=object)], [array([2276.67, 2320.0, 2140.0, ..., 2300.0, 2393.33, 2410.0],
      dtype=object), array([2070.0, 2286.67, 2176.67, ..., 2080.0, 2106.67, 2150.0],
      dtype=object), array([1456.67, 1293.33, 1456.67, ..., 2393.33, 2386.67, 2496.67],
      dtype=object), array([1010.0, 1095.0, 1220.0, ..., 1050.0, 1135.0, 1080.0], dtype=object)]]


AttributeError: 'list' object has no attribute 'tolist'